# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2 toc-item"><a href="#Generate-List-of-Data" data-toc-modified-id="Generate-List-of-Data-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Generate List of Data</a></div><div class="lev2 toc-item"><a href="#Construct-Configuration-Matrices" data-toc-modified-id="Construct-Configuration-Matrices-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Construct Configuration Matrices</a></div><div class="lev1 toc-item"><a href="#Detect-Dynamic-Subgraphs" data-toc-modified-id="Detect-Dynamic-Subgraphs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Detect Dynamic Subgraphs</a></div><div class="lev2 toc-item"><a href="#Map-NMF-Consensus-to-Identify-Seed-Subgraphs" data-toc-modified-id="Map-NMF-Consensus-to-Identify-Seed-Subgraphs-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Map NMF Consensus to Identify Seed Subgraphs</a></div><div class="lev2 toc-item"><a href="#Consensus-Clustering-of-Dynamic-Subgraphs" data-toc-modified-id="Consensus-Clustering-of-Dynamic-Subgraphs-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Consensus Clustering of Dynamic Subgraphs</a></div><div class="lev3 toc-item"><a href="#Plot-Subgraphs" data-toc-modified-id="Plot-Subgraphs-221"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Plot Subgraphs</a></div><div class="lev1 toc-item"><a href="#Test-Retest-Reliability" data-toc-modified-id="Test-Retest-Reliability-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Test-Retest Reliability</a></div><div class="lev1 toc-item"><a href="#Detect-Geometric-Null-Subgraphs" data-toc-modified-id="Detect-Geometric-Null-Subgraphs-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Detect Geometric-Null Subgraphs</a></div><div class="lev2 toc-item"><a href="#Generate-Geometric-Adjacency-Matrix" data-toc-modified-id="Generate-Geometric-Adjacency-Matrix-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Generate Geometric Adjacency Matrix</a></div><div class="lev2 toc-item"><a href="#Run-Non-Negative-Matrix-Factorization-Algorithm" data-toc-modified-id="Run-Non-Negative-Matrix-Factorization-Algorithm-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Run Non-Negative Matrix Factorization Algorithm</a></div><div class="lev3 toc-item"><a href="#Plot-Subgraphs" data-toc-modified-id="Plot-Subgraphs-421"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Plot Subgraphs</a></div>

# Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
import sys
import glob

import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.io as io
import h5py
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

sys.path.append('/Users/akhambhati/Developer/hoth_research/Echobase')
import Echobase
convert_conn_vec_to_adj_matr = Echobase.Network.Transforms.configuration.convert_conn_vec_to_adj_matr
convert_adj_matr_to_cfg_matr = Echobase.Network.Transforms.configuration.convert_adj_matr_to_cfg_matr
nmf = Echobase.Network.Partitioning.Subgraph.nmf

rcParams = Echobase.Plotting.fig_format.update_rcparams(rcParams)

path_CoreData = '/Users/akhambhati/Remotes/CORE.fMRI_multiband.mmattar/restdata'
path_PeriphData = '/Users/akhambhati/Remotes/RSRCH.NMF_Subnetworks'
path_AtlasData = '/Users/akhambhati/Remotes/CORE.MRI_Atlases'
path_InpData = path_PeriphData + '/e01d-Dyne_FuncNetw'
path_ExpData = path_PeriphData + '/e03d-DynFuncSubgraph-Population'

for path in [path_CoreData, path_PeriphData, path_InpData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)
        
if not os.path.exists('./e03d-Figures'):
    os.makedirs('./e03d-Figures')        

## Generate List of Data

In [ ]:
subj_date = [full_subj_path.split('/')[-1]
             for full_subj_path in glob.iglob('{}/Adjacency.*.npz'.format(path_InpData))]

subj_ids = {}
for s_d in subj_date:
    subj, date = s_d.split('.')[1:3]
    try:
        subj_ids[subj]
    except KeyError:
        subj_ids[subj] = []
    
    subj_ids[subj].append(date)

## Construct Configuration Matrices
*__WARNING: Will Delete Existing Output__*

In [ ]:
# Remove all existing output (retains pipe/pipeline definitions)
rm_outp = glob.glob("{}/NMF_Optimization.CfgMatr.npz".format(path_ExpData))

for rm_type in [rm_outp]:
    for path in rm_type:
        try:
            os.remove(path)
        except:
            print("{} not found".format(path))

In [ ]:
cfg_matr = []
cfg_name = []
for df_name in subj_date:
    
    # Read the input data
    df = np.load('{}/{}'.format(path_InpData, df_name))

    for cfg_vec in convert_adj_matr_to_cfg_matr(df['adj_matr']):
        cfg_matr.append(cfg_vec)
        cfg_name.append('.'.join(df_name.split('.')[1:3]))

# Cache the configuration matrices
cfg_matr = np.array(cfg_matr)
cfg_name = np.array(cfg_name)
np.savez('{}/NMF_Optimization.CfgMatr.npz'.format(path_ExpData),
         cfg_matr=cfg_matr, cfg_name=cfg_name)

# Detect Dynamic Subgraphs

## Map NMF Consensus to Identify Seed Subgraphs
*__WARNING: Will Delete Existing Output__*

In [ ]:
def find_seed_subgraphs(proc_item):
    path_Output = '{}/NMF_Consensus.Param.{}.npz'.format(path_ExpData, proc_item['seed'])

    # Check if the output already exists (can be commented if overwrite is needed)
    if os.path.exists(path_Output):
        raise Exception('Output {} already exists'.format(path_Output))

    # Load the configuration matrix and optimal parameter set
    cfg_data = np.load('{}/NMF_Optimization.CfgMatr.npz'.format(path_ExpData))
    cfg_matr = cfg_data['cfg_matr']

    # Grab the task ID of the current job (and the associated parameter dictionary)
    fac_subnet = np.random.uniform(low=0, high=1.0,
                                   size=(proc_item['rank'],
                                         cfg_matr.shape[1]))
    fac_coef = np.random.uniform(low=0, high=1.0,
                                 size=(proc_item['rank'],
                                       cfg_matr.shape[0]))

    # Run NMF Algorithm
    fac_subnet, fac_coef, err = Echobase.Network.Partitioning.Subgraph.nmf.snmf_bcd(
        cfg_matr,
        alpha=proc_item['alpha'],
        beta=proc_item['beta'],
        fac_subnet_init=fac_subnet,
        fac_coef_init=fac_coef,
        max_iter=100, verbose=False)

    # Cache the NMF result
    np.savez(path_Output,
             fac_subnet=fac_subnet,
             fac_coef=fac_coef,
             err=err)
    
    print(path_Output)

In [ ]:
param = {'rank': 21,
         'alpha': 0.50,
         'beta': 0.39}
n_seed = 100

# Generate a processing joblist
proc_list = []
for seed in xrange(n_seed):
    proc_list.append({'rank': param['rank'],
                      'alpha': param['alpha'],
                      'beta': param['beta'],
                      'seed': seed+1})

from multiprocessing import Pool
parallel_run = True

if parallel_run:
    mp = Pool(7)
    mp.map(find_seed_subgraphs, proc_list)
else:
    find_seed_subgraphs(proc_list[0])
    #map(find_seed_subgraphs, proc_list)

## Consensus Clustering of Dynamic Subgraphs

In [ ]:
seed_paths = glob.glob("{}/NMF_Consensus.Param.*.npz".format(path_ExpData))

# Aggregate the estimated subgraphs of each seed
fac_subnet_seeds = []
for ii, path in enumerate(seed_paths):
    data = np.load(path, mmap_mode='r')
    fac_subnet = data['fac_subnet'][:, :]
    data.close()

    n_fac = fac_subnet.shape[0]
    n_conn = fac_subnet.shape[1]

    for iy in xrange(fac_subnet.shape[0]):
        fac_subnet_seeds.append(fac_subnet[iy, :])
fac_subnet_seeds = np.array(fac_subnet_seeds)

n_obs = fac_subnet_seeds.shape[0]
n_conn = fac_subnet_seeds.shape[1]

# Consensus Subgraphs
fac_cons_subnet, fac_cons_seeds, err = nmf.snmf_bcd(
    fac_subnet_seeds,
    alpha=0.0,
    beta=0.0,
    fac_subnet_init=np.random.uniform(low=0.0, high=1.0, size=(n_fac, n_conn)),
    fac_coef_init=np.random.uniform(low=0.0, high=1.0, size=(n_fac, n_obs)),
    max_iter=100, verbose=False)

# Consensus Coefficients
cfg_matr_path = glob.glob("{}/NMF_Optimization.CfgMatr.npz".format(path_ExpData))[0]
data_cfg = np.load(cfg_matr_path, mmap_mode='r')
n_win = data_cfg['cfg_matr'].shape[0]
fac_cons_subnet_2, fac_cons_coef_2, err = nmf.snmf_bcd(
    data_cfg['cfg_matr'],
    alpha=0.0,
    beta=0.0,
    fac_subnet_init=fac_cons_subnet,
    fac_coef_init=np.random.uniform(low=0.0, high=1.0, size=(n_fac, n_win)),
    max_iter=100, verbose=False)

# Cache the Consensus NMF result
np.savez("{}/NMF_Optimization.consensus_subgraph.npz".format(path_ExpData),
         fac_subnet=fac_cons_subnet_2, fac_coef=fac_cons_coef_2, err=err)

### Plot Subgraphs

In [ ]:
%matplotlib inline

# Load the consensus data
data = np.load("{}/NMF_Optimization.consensus_subgraph.npz".format(path_ExpData),
               mmap_mode='r')
fac_subnet = data['fac_subnet']
fac_coef = data['fac_coef']

# Normalize
fac_subnet = fac_subnet / fac_subnet.max()
fac_coef = fac_coef / fac_coef.max()

n_fac = fac_subnet.shape[0]
n_conn = fac_subnet.shape[1]
n_win = fac_coef.shape[1]

# Plot subgraph matrix
plt.figure()
ax = plt.subplot(111)
mat = ax.matshow(fac_subnet.T, aspect=n_fac/n_conn, cmap='rainbow', vmin=0, vmax=1)
plt.colorbar(mat, ax=ax)

ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
#ax.set_xticks(np.linspace(0, 80, 5))
ax.set_ylabel('Functional Interactions')
ax.set_xlabel('Subgraphs')

plt.savefig('./e03d-Figures/Subgraph-Cfg_Matrix.svg')
plt.close()      

# Plot subgraph adjacency
plt.figure()
n_row = np.floor(np.sqrt(n_fac))
n_col = np.ceil(n_fac / n_row)
for ii, subg in enumerate(fac_subnet):
    adj = convert_conn_vec_to_adj_matr(subg)

    ax = plt.subplot(n_row, n_col, ii+1)
    mat = ax.matshow(adj, cmap='rainbow', vmin=0, vmax=1)
    #plt.colorbar(mat, ax=ax)
    ax.set_axis_off()
    
plt.savefig('./e03d-Figures/Subgraph-Adj_Matrices.svg')
plt.show()
plt.close()      

# Plot Coefficients
plt.figure()
ax = plt.subplot(111)

fac_coef = fac_coef.T
norm_fac = fac_coef - fac_coef.mean(axis=0)
for ff in xrange(n_fac):
    ax.plot(ff + norm_fac[:, ff] / (3*np.std(norm_fac[:, ff])), color=[66/256., 152/256., 221./256])

# Axis Settings
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylim([-1, n_fac+1])
ax.set_ylabel('Subgraphs')
ax.set_xlabel('Time Windows')

plt.savefig('./e03d-Figures/Subgraph-Coefs.svg')
plt.show()
plt.close()  

# Test-Retest Reliability

In [ ]:
import scipy.optimize as sciopt
import string

mod_ord_ii = np.array([ 7, 19, 10, 12,  5,  3,  0, 11,  6, 16, 15,  8, 14, 18, 13,  1,  4,
                       9,  2, 17, 20])
subg_letter = np.array(list(string.ascii_lowercase))[:21]

data_ds1 = np.load("{}/NMF_Optimization.consensus_subgraph.npz".format('/Users/akhambhati/Remotes/RSRCH.NMF_Subnetworks/e03b-DynFuncSubgraph-Population'),
                     mmap_mode='r')
ds1_fac_subnet = data_ds1['fac_subnet']
ds1_fac_subnet = ds1_fac_subnet[mod_ord_ii, :]

data_ds2 = np.load("{}/NMF_Optimization.consensus_subgraph.npz".format(path_ExpData),
                     mmap_mode='r')
ds2_fac_subnet = data_ds2['fac_subnet']

n_fac = ds1_fac_subnet.shape[0]

def hungarian(ds1_fac_subnet, ds2_fac_subnet):
    n_fac = ds1_fac_subnet.shape[0]
    cost_matrix = np.zeros((n_fac, n_fac))

    for ds1_fac_ii in xrange(ds1_fac_subnet.shape[0]):
        for ds2_fac_ii in xrange(ds2_fac_subnet.shape[0]):
            cost_matrix[ds1_fac_ii, ds2_fac_ii] = np.linalg.norm(ds1_fac_subnet[ds1_fac_ii] -
                                                                 ds2_fac_subnet[ds2_fac_ii])

    ds1_old_ii, ds1_new_ii = sciopt.linear_sum_assignment(cost_matrix)
    ds1_fac_subnet[ds1_new_ii, :] = ds1_fac_subnet[ds1_old_ii, :]

    val = np.array([stats.pearsonr(ds1_fac_subnet[fac_i, :], ds2_fac_subnet[fac_i, :])
                    for fac_i in xrange(n_fac)])
    
    r_value, p_value = val[:, 0], val[:, 1]
    
    return r_value, p_value, ds1_fac_subnet, ds2_fac_subnet


true_rho, true_pv, fac_A, fac_B = hungarian(ds1_fac_subnet, ds2_fac_subnet)
fac_A = fac_A[np.argsort(true_rho)[::-1]]
fac_B = fac_B[np.argsort(true_rho)[::-1]]

alpha = 0.05
beta = alpha/len(true_rho)

plt.figure(dpi=300)
ax = plt.subplot(111)

clr = []
for ix in np.argsort(true_rho)[::-1]:
    if true_pv[ix] < beta:
        clr.append('b')
    else:
        clr.append('k')
        
ax.bar(xrange(n_fac), np.sort(true_rho)[::-1], color=clr, lw=0)

ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Frequency')
ax.set_xlabel('Subgraph Correlation')
ax.set_xlim([0, 21])
ax.set_ylim([-0.3, 1.0])
plt.savefig('./e03d-Figures/Test_Retest.svg')
plt.show()

In [ ]:
%matplotlib inline

# Plot subgraph adjacency
fig = plt.figure(figsize=(12, 12), dpi=300)
for ii, (fa, fb) in enumerate(zip(fac_A, fac_B)):
    adj_A = convert_conn_vec_to_adj_matr(fa)
    adj_B = convert_conn_vec_to_adj_matr(fb)    
    
    adj_AB = np.nan*np.zeros((2*adj_A.shape[0]+20, 2*adj_A.shape[0]+20))
    adj_AB[:adj_A.shape[0], :adj_A.shape[0]] = adj_A
    adj_AB[:adj_A.shape[0], -adj_A.shape[0]:] = adj_B
    
    ax = fig.add_subplot(7, 3, ii+1)
    mat = ax.matshow(adj_AB, cmap='viridis')
    ax.set_axis_off()
    ax.set_title('Subgraph: {}'.format(ii+1))
    
fig.tight_layout(pad=0.1, h_pad=0.1, w_pad=0.1)
fig.savefig('./e03d-Figures/Paired_Subgraphs.svg')
fig.show()

# Detect Geometric-Null Subgraphs

## Generate Geometric Adjacency Matrix

In [ ]:
# Generate Node Distance Matrix
df = pd.read_csv('{}/Atlas/HOA112_Labels.csv'.format(path_CoreData))
n_node = len(df)

dist_matr = np.zeros((n_node, n_node))
ix, iy = np.mgrid[:n_node, :n_node]

dX = np.array(df.X)
dY = np.array(df.Y)
dZ = np.array(df.Z)

dist_matr[ix, iy] = np.sqrt((dX[ix]-dX[iy])**2 + 
                            (dY[ix]-dY[iy])**2 +
                            (dZ[ix]-dZ[iy])**2)
#dist_matr /= dist_matr.max()

np.savez('{}/geom_adj_matr.npz'.format(path_ExpData),
         dist_matr=dist_matr)

## Run Non-Negative Matrix Factorization Algorithm
*__WARNING: Will Delete Existing Output__*

In [ ]:
# Remove all existing output (retains pipe/pipeline definitions)
rm_outp_geomnull = glob.glob("{}/NMF_GeomNull.subgraph_seed-*.npz".format(path_ExpData))
rm_outp_geomnull = glob.glob("{}/NMF_EdgeNull.subgraph_seed-*.npz".format(path_ExpData))

for rm_type in [rm_outp]:
    for path in rm_type:
        try:
            os.remove(path)
        except:
            print("{} not found".format(path))

In [ ]:
param = {'rank': 21,
         'alpha': 0.50,
         'beta': 0.39}
n_seed = 1000

from multiprocessing import Pool
parallel_run = True

# Generate a processing joblist
cfg_matr_path = glob.glob("{}/NMF_Optimization.CfgMatr.npz".format(path_ExpData))[0]
proc_list = []
for seed in xrange(n_seed):
    proc_list.append({'path': cfg_matr_path,
                      'param': param,
                      'seed': seed+1,
                      'dist_path': '{}/geom_adj_matr.npz'.format(path_ExpData)})
    
# Setup helper function to map pipeline run
def _nmf_helper(proc_item):
    
    # Load the file
    #if os.path.exists(inp_path):
    #    return 0
    print(" -- Processing Seed: {}".format(proc_item['seed']))
    data = np.load(proc_item['path'], mmap_mode='r')
    dist_matr = np.load(proc_item['dist_path'])['dist_matr']
    
    # Generate a geometric null topology from cfg_matr
    cfg_matr_edge_null = []
    cfg_matr_geom_null = []
    for vec_i, cfg_vec in enumerate(data['cfg_matr']):
        if vec_i == 0:
            perm_seq = np.random.permutation(len(cfg_vec))
        adj_matr = convert_conn_vec_to_adj_matr(cfg_vec)        
        adj_null, adj_dtr = Echobase.Network.Rewire.geometry.surrogate_trend(adj_matr, dist_matr, 3, 3, perm_seq=perm_seq)
        cfg_matr_edge_null.append(convert_adj_matr_to_cfg_matr(np.expand_dims(adj_null, axis=0)).reshape(-1))
        cfg_matr_geom_null.append(convert_adj_matr_to_cfg_matr(np.expand_dims(adj_dtr, axis=0)).reshape(-1))
    cfg_matr_edge_null = np.array(cfg_matr_edge_null)        
    cfg_matr_geom_null = np.array(cfg_matr_geom_null)

    for null_name, cfg_matr_null in [('GeomNull', cfg_matr_geom_null),
                                     ('EdgeNull', cfg_matr_edge_null)]:
        # Initialize the factors for NMF
        fac_subnet = np.random.uniform(low=0, high=1.0,
                                       size=(proc_item['param']['rank'],
                                             cfg_matr_null.shape[1]))
        fac_coef = np.random.uniform(low=0, high=1.0,
                                     size=(proc_item['param']['rank'],
                                           cfg_matr_null.shape[0]))

        # Run NMF Algorithm
        fac_subnet, fac_coef, err = nmf.snmf_bcd(
            cfg_matr_null,
            alpha=proc_item['param']['alpha'],
            beta=proc_item['param']['beta'],
            fac_subnet_init=fac_subnet,
            fac_coef_init=fac_coef,
            max_iter=100, verbose=False)

        # Cache the NMF result
        np.savez("{}/NMF_{}.subgraph_seed-{}.npz".format(path_ExpData,
                                                         null_name,
                                                         proc_item['seed']),
                 fac_subnet=fac_subnet, fac_coef=fac_coef, err=err,
                 param=proc_item['param'], path=proc_item['path'])

if parallel_run:
    mp = Pool(7)
    mp.map(_nmf_helper, proc_list)
else:
    map(_nmf_helper, proc_list)

### Plot Subgraphs

In [ ]:
for null_type in ['EdgeNull', 'GeomNull']:
    seed_paths = glob.glob("{}/NMF_{}.subgraph_seed-*.npz".format(path_ExpData, null_type))
    data = np.load(np.random.permutation(seed_paths)[0])

    fac_subnet = data['fac_subnet']
    fac_coef = data['fac_coef']

    # Normalize
    fac_subnet = fac_subnet / fac_subnet.max()
    fac_coef = fac_coef / fac_coef.max()

    n_fac = fac_subnet.shape[0]
    n_conn = fac_subnet.shape[1]
    n_win = fac_coef.shape[1]

    # Plot subgraph adjacency
    plt.figure()
    n_row = np.floor(np.sqrt(n_fac))
    n_col = np.ceil(n_fac / n_row)
    for ii, subg in enumerate(fac_subnet):
        adj = convert_conn_vec_to_adj_matr(subg)

        ax = plt.subplot(n_row, n_col, ii+1)
        mat = ax.matshow(adj, cmap='rainbow', vmin=0, vmax=1)
        ax.set_axis_off()

    plt.savefig('./e03d-Figures/{}-Subgraph-Adj_Matrices.svg'.format(null_type))
    plt.show()
    plt.close()      

    # Plot Coefficients
    plt.figure()
    ax = plt.subplot(111)

    fac_coef = fac_coef.T
    norm_fac = fac_coef - fac_coef.mean(axis=0)
    for ff in xrange(n_fac):
        ax.plot(ff + norm_fac[:, ff] / (3*np.std(norm_fac[:, ff])), color=[66/256., 152/256., 221./256])

    # Axis Settings
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_ylim([-1, n_fac+1])
    ax.set_ylabel('Subgraphs')
    ax.set_xlabel('Time Windows')

    plt.savefig('./e03d-Figures/{}-Subgraph-Coefs.svg'.format(null_type))
    plt.show()
    plt.close()  